<a href="https://colab.research.google.com/github/MissCrispenCakes/GPT2_example_CAMEROON/blob/master/Lab_10_GPT_2_cameroon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2 Example

## Here we will run through the popular GPT-2:

GPT-2 is a neural-network-powered language model. A language model is a model that predicts the likelihood of a sentence existing in the world. For example, a language model can label the sentence “I take my dog for a walk” as more probable to exist (i.e. on the Internet) than the sentence “I take my banana for a walk.” This is true for sentences as well as phrases and, more generally, any sequence of characters.

Like most language models, GPT-2 is elegantly trained on an unlabeled text dataset (in this case, the training data includes among others Common Crawl and Wikipedia). Words or phrases are randomly removed from the text, and the model must learn to fill them in using only the surrounding words as context. It’s a simple training task that results in a powerful and generalizable model.


---




Setup:

1) Make sure GPU is enabled, go to edit->notebook settings->Hardware Accelerator GPU

2) Make a copy to your google drive, click on copy to drive in panel

Note: Colab will reset after 12 hours make sure to save your model checkpoints to google drive around 10-11 hours mark or before, then go to runtime->reset all runtimes. Now copy your train model back into colab and start training again from the previous checkpoint.

clone and cd into repo


---
# FOR NOW I AM USING MY PUBLIC REPO CLONE - WE CAN CLONE INTO THE CAMEROON GITHUB>?


---




In [ ]:
!git clone https://github.com/MissCrispenCakes/GPT2_example_CAMEROON.git

Cloning into 'GPT2_example_CAMEROON'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 48 (delta 22), reused 10 (delta 4), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [ ]:
cd GPT2_example_CAMEROO\N

/content/GPT2_example_CAMEROON


Install requirements.

We will use an earlier version of TensorFlow than we have been using for our labs so far - this will make it easier for you to play around with the introductory content that exists online. (If time will update/transfer to newer TF - don't worry about the 'incompatible' flags - they get resolved)

In [ ]:
!pip3 install tensorflow==1.15.0rc3
!pip3 install -r requirements.txt

     |████████████████████████████████| 412.3MB 35kB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 3.8MB 50.2MB/s 
     |████████████████████████████████| 512kB 49.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=01c4abeb5686699751be2f036ac69cf24d40750ed0da4be5d97a2cac8bb79f4e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

Mount drive to access google drive for saving and accessing checkpoints later. Have to log in to your google account - you may already be connected from Lab 9, run this anyway to check.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download the model data - we will only consider pre-trained models that are small and medium in size (due to time, space)

*   117M
*   345M



In [ ]:
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 862kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 62.4Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 855kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:06, 82.2Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 4.97Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 45.8Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 43.4Mit/s]                                                       


In [ ]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 1.05Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 66.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.40Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:16, 85.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 11.1Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 61.8Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 60.5Mit/s]                                                       


encoding

In [ ]:
!export PYTHONIOENCODING=UTF-8

Fetch checkpoints if you have them saved in google drive -  if you don't have a saved checkpoint yet DON'T worry! The following commands will still run and check the drive but won't find anything yet.

In [ ]:
!cp -r /content/drive/My\Drive/checkpoint/ /content/GPT2_example_CAMEROO\N// 

cp: cannot stat '/content/drive/MyDrive/checkpoint/': No such file or directory



Let's get our train on! In this case the file is A Tale of Two Cities (Charles Dickens) from Project Gutenberg. To change the dataset GPT-2 models will fine-tune on, change this URL to another .txt file, and change corresponding part of the next cell. Note that you can use small datasets if you want but you will have to be sure not to run the fine-tuning for too long or you will overfit badly. Roughly, expect interesting results within minutes to hours in the 1-10s of megabyte ballpark, and below this you may want to stop the run early as fine-tuning can be very fast.

In [ ]:
!wget https://www.gutenberg.org/files/98/98-0.txt

--2020-12-30 21:56:59--  https://www.gutenberg.org/files/98/98-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 807615 (789K) [text/plain]
Saving to: ‘98-0.txt’

98-0.txt            100%[===================>] 788.69K  2.01MB/s    in 0.4s    

2020-12-30 21:56:59 (2.01 MB/s) - ‘98-0.txt’ saved [807615/807615]




Start training, add --model_name '345M' to use 345 model. Use 117M for the smaller model. 

In [ ]:
!chmod u+rwx train.py
!PYTHONPATH=src ./train.py --dataset /content/GPT2_example_CAMEROO\N//98-0.txt --model_name '345M'


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2020-12-30 21:56:22.679014: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-12-30 21:56:22.683538: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-12-30 21:56:22.683722: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2955640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-12-30 21:56:22.683752: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-12-30 21:56:22.687516

Save our checkpoints to start training again later

In [ ]:
!cp -r /content/GPT2_example_CAMEROO\N/checkpoint /content/drive/My\ Drive/

Load your trained model for use in sampling below (117M or 345M) -  if you don't have a trained model yet DON'T worry! The following commands will still run and check the drive but won't find anything yet.

In [ ]:
!cp -r /content/GPT2_example_CAMEROO\N/checkpoint/run1/* /content/GPT2_example_CAMEROO\N/models/117M/

cp: cannot stat '/content/GPT2_example_CAMEROON/checkpoint/run1/*': No such file or directory


In [ ]:
!cp -r /content/GPT2_example_CAMEROO\N/checkpoint/run1/* /content/GPT2_example_CAMEROO\N/models/345M/

cp: cannot stat '/content/GPT2_example_CAMEROON/checkpoint/run1/*': No such file or directory


Generate conditional samples from the model given a prompt you provide -  change top-k hyperparameter if desired (default is 40),  if you're using 345M, add "--model-name 345M"

# Have fun with the model inputs! Try words, sentences, paragraphs!

In [ ]:
!chmod u+rwx src/interactive_conditional_samples.py
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "345M"



2020-12-30 21:57:27.749906: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-30 21:57:27.788243: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-30 21:57:27.788874: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-12-30 21:57:27.789209: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-12-30 21:57:27.790344: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-12-30 21:57:27.791458: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0

To check flag descriptions, use:

In [ ]:
!python3 src/interactive_conditional_samples.py -- --help

Generate unconditional samples from the model,  if you're using 345M, add "--model-name 345M"

In [ ]:
!chmod u+rwx src/generate_unconditional_samples.py
!python3 src/generate_unconditional_samples.py --model_name "345M" | tee /tmp/samples

To check flag descriptions, use:

In [ ]:
!python3 src/generate_unconditional_samples.py -- --help